# Introduction to Brightway2
This is a notebook prepared for the course on [Green Engineering](https://guies.uab.cat/guies_docents/public/portal/html/2024/assignatura/44729/es), part of the [RICSE](https://www.uab.cat/web/estudiar/official-master-s-degrees/general-information-1096480962610.html?param1=1345875382068) master program at UAB. It is prepared by Cristina Madrid (cristina.madrid@uab.cat) using material from the Brightway2 documentation and the [Brightway Book](https://learn.brightway.dev/en/latest/content/chapters/BW25/BW25_introduction.html). We will be working with BW 2.5 (Note that the BW version you installed for Activity Browser is not the same. You need to install the correct version for this course).

As indicated in the [BW2 installation site](https://docs.brightway.dev/en/latest/content/installation/index.html), you first need to install a distribution of python and [miniconda](https://docs.anaconda.com/miniconda/) is recommended.

 After you have python (miniconda) installed, we need to create a dedicated python environment and install Brightway 2.5 and other necessary packages. The Brightway community has created a file with information to download a Conda environment including these packages here: [env_bw25.yml](https://github.com/brightway-lca/brightway-book/raw/refs/heads/main/content/chapters/BW25/_environment/env_bw25.yml). You can install them using the command:

In [1]:
conda env create -f env_bw25.yml --solver libmamba

ValueError: The python kernel does not appear to be a conda environment.  Please use ``%pip install`` instead.

Now you need to "activate" (import) the needed packages:

In [ ]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

After you have BW25 up and running, you need to understand the BW2 data structure. Please read about it in the BW25 site [here](https://docs.brightway.dev/en/latest/content/overview/structure.html).


# 1. Creating an LCA tree model

Here is an exercise to help you understand how to create a basic supply chain model in Brightway2. We will create a simple graph, and then calculate the LCA results for it. The goal is to understand how to create nodes and edges, and how to calculate LCA results.

In [1]:
import bw2data as bd
import pandas as pd

The first thing to learn about `bw2data` is the concept of projects. Each project is self-contained, and independent of other projects. Each has its own subdirectory. This can lead to data duplication, but helps keep each project safe from the changes in the others.

We start in the `default` project:

In [2]:
bd.projects.current

'default'

It has its own directory:

In [3]:
bd.projects.dir

PosixPath('/home/jupyter-as23_cristina.madr-c601e/.local/share/Brightway3/default.c21f969b')

Make it easy to reset this project:

In [4]:
try:
    bd.projects.delete_project("supply chain graph", True)
except ValueError:
    pass

Let's create a new project:

In [5]:
bd.projects.set_current("supply chain graph")

A graph can have nodes of any type, but for the purposes of LCA it is convenient to separate activity nodes, product nodes, elementary flow nodes, and characterization nodes. We will also use a shortcut (for now) and assume that each activity produces exactly one product, so we can collapse activity and products nodes into a single node.

Let's think about a simple product system - a bike. Here is a graph:

To enter this data into BW, we need to create the nodes, and then the edges. We will create these nodes in a `Database`. A database in BW is just a collection of nodes - it can be large or small, there aren't any general rules.

In [6]:
db = bd.Database("bike")
# Let the metadata system know this database exists. Not necessary if using a `bw2io` importer.
db.register()

Our first node. We specify some additional data to make it easier to find or use this node later on.

In [7]:
data = {
    'code': 'bike',
    'name': 'bike production',
    'location': 'DK',
    'unit': 'bike'
}

bike = db.new_node(**data)
bike.save()

In [8]:
data = {
    'code': 'ng',
    'name': 'natural gas production',
    'location': 'NO',
    'unit': 'MJ'
}

ng = db.new_node(**data)
ng.save()

In [9]:
data = {
    'code': 'cf',
    'name': 'carbon fibre production',
    'location': 'DE',
    'unit': 'kg'
}

cf = db.new_node(**data)
cf.save()

We can also define nodes like this. Note that this node has a `type`.

In [10]:
co2 = db.new_node(
    code='co2', 
    name="Carbon Dioxide", 
    categories=('air',),
    type='emission',
    unit='kg'
)

co2.save()

We also need to create edges between the nodes. We can do this in many ways, here is one:

In [11]:
bike.new_edge(
    amount=2.5, 
    type='technosphere',
    input=cf
).save()

What about some uncertainty? We use [stats_arrays](https://stats-arrays.readthedocs.io/en/latest/) to model probability distribution functions.

In [12]:
cf.new_edge(
    amount=237.3,  # plus 58 kWh of electricity, in ecoinvent 3.8 
    uncertainty_type=5, 
    minimum=200, 
    maximum=300, 
    type='technosphere',
    input=ng,
).save()

In [13]:
cf.new_edge(
    amount=26.6, 
    uncertainty_type=5, 
    minimum=26,
    maximum=27.2, 
    type='biosphere',
    input=co2,
).save()

Brightway will assume that the a `production` exchange of amount 1 in each node unless you tell it otherwise.

# Exercise

Create a new bicycle made of steel. You will need some coal and some iron ore mining for the steel, and some steel for the bicycle.

Your use of steel consumed some iron ore, a natural resource. In a **new database**, create a biosphere flow for this iron ore, and add the iron ore flow. You might not be sure about the numbers - you can reflect that in the uncertainty you assign to the exchanges.

## LCIA

To define characterization nodes and edges, we use a different data structure:

In [14]:
ipcc = bd.Method(('IPCC',))
ipcc.write([
    (co2.key, {'amount': 1, 'uncertainty_type': 3, 'loc': 1, 'scale': 0.05}),
])

This is already enough to do an LCA. Let's check what our answer should be. Without uncertainty, to make one bike we need 2.5 kg of CF, and carbon fibre produces 26.6 kg CO2 per kg of carbon fibre, so we are looking for around 60 kg CO2-eq.

In [15]:
import bw2calc as bc

In [16]:
lca = bc.LCA(demand={bike: 1}, method=('IPCC',))
lca.lci()
lca.lcia()
lca.score

66.50000095367433

To use uncertainty, we tell the `LCA` object to use the probability distributions:

In [ ]:
lca = bc.LCA(demand={bike: 1}, method=('IPCC',), use_distributions=True)
lca.lci()
lca.lcia()

df = pd.DataFrame([{'score': lca.score} for _ in zip(lca, range(10))])
df

## Exercise

Create a new LCIA method for your iron ore consumption. Calculate the LCIA result you should get, and then verify that you have the correct value.

In [17]:
ore = db.new_node(
    code='ore', 
    name="Ore", 
    categories=('earth',),
    type='input',
    unit='kg'
)
ore.save()

In [19]:
bike.new_edge(
    amount=2.5, 
    type='technosphere',
    input=ore,
    uncertainty_type=5, 
    minimum=200, 
    maximum=300
).save()

In [20]:
list(bike.technosphere())

[Exchange: 2.5 kg 'carbon fibre production' (kg, DE, None) to 'bike production' (bike, DK, None)>,
 Exchange: 2.5 kg 'Ore' (kg, GLO, ('earth',)) to 'bike production' (bike, DK, None)>,
 Exchange: 2.5 kg 'Ore' (kg, GLO, ('earth',)) to 'bike production' (bike, DK, None)>]

In [ ]:
co2.save()CrisMethod=bd.Method(('CrisMethod',))
CrisMethod.write([
    (ore.key, {'amount': 1, 'uncertainty:type':3, 'loc':1, 'scale':0.05}),
])

lca= bc.LCA(demand ={bike:1}

## Exercise

Open a new notebook, and calculate an LCIA score for you steel bike using everything you have learned already. You are not allowed to duplicate this notebook :) But you can copy and paste individual cells.

## Searching through the database

In addition to storing and using nodes and edges, our graph database can be searched in different ways. Let's show this with a larger database.

We can use a shortcut to install some data:

In [ ]:
import bw2io as bi
bi.install_project("USEEIO-1.1")

# If that doesn't work for whatever reason, we can import the original data with this:
# bi.useeio11()

In [ ]:
bd.projects.set_current("USEEIO-1.1")

In [ ]:
bd.databases

In [ ]:
db = bd.Database("USEEIO-1.1")
db.name

This database has processes **and** products. What would this mean for drawing a graph?

In [ ]:
{node['type'] for node in db}

We can search with the 'search' function.

In [ ]:
fun = db.search("amusement")[0]
fun['name'] = 'fun'
fun.save()

In [ ]:
db.search('amusement')

In [ ]:
{node['name'] for node in db if node['type'] == 'product'}

## Interacting with the graph

In [ ]:
moo = bd.get_node(name='Cattle ranches and feedlots', type='product')

In [ ]:
type(moo) == bd.Node

We can assign any attributes to nodes (and to edges)

In [ ]:
moo['moo'] = 'loud'

Note that this attributes are not saved to the database by default - we have to tell Brightway to save changed data!

We have some attributes which are common to all inventory databases

In [ ]:
moo['categories'], moo['location'], moo['unit']

The node classes act like dictionaries, and raise error for missing keys

In [ ]:
moo['missing']

## Exercise

Iterate through all `product` nodes in the US EEIO and tag every node whose combined name and unit is more than 40 characters long with `"long" = True`.

## Exchange iterators

We have shortcuts to traverse the supply chain graph. For inputs, we have `.technosphere()` and `.biospere()`; for producing edges, we have `.producers()`, and for edges to other nodes which consume the outputs of our node there is `.consumers()`. You can also get all edges with `.edges()`.

These are all [iterators](https://jakevdp.github.io/WhirlwindTourOfPython/10-iterators.html).

In [ ]:
bd.projects.set_current("supply chain graph")

In [ ]:
list(cf.technosphere())

In [ ]:
list(cf.biosphere())

In [ ]:
list(cf.consumers())

If no edge with the type `production` is given, Brightway will assume that an activity produces one unit of its reference product, so sometimes the production edges are not given explicitly.

In [ ]:
list(cf.producers())

The US EEIO is the same, it is normalized to the production of one USD. It can be interesting to sum the costs of the inputs:

In [ ]:
bd.projects.set_current("USEEIO-1.1")

In [ ]:
sum([o['amount'] for o in bd.get_node(name='Cattle ranches and feedlots', type='process').technosphere()])

# Contribution

Let's show a little bit of what Brightway can do. We can compare the correlation of LCA scores across a variety of categories.

There is an atuomatic way to do this in Brightway, but we can also program it manually to see how it works.

Stop for a bit and think about what one would need to calculate LCA scores for 380 products and ~10 impact categories.

In [ ]:
products_in_order = [obj for obj in db if obj['type'] == 'product']
categories_in_order = [method for method in bd.methods if method[0] == 'Impact Potential']

In general, in Brightway there is *one secret* to getting good performance: Don't rebuild matrices unless you really need to. Rebuilding a matrix is not slow, but the time adds up if you do it a lot. But most importantly, if you are using `pypardiso` (normally everything except for ARM machines), and you keep the technosphere matrix the same, it will secretly remember all the preparation work it did to solve the linear system and you will get a factor of 100 speed increase on subsequent calculations.

So, in this case we will use one `LCA` object, and use the `lci` method repeatedly.

In [ ]:
import numpy as np

results = np.zeros((len(products_in_order), len(categories_in_order)))

def get_lcia_scores(products, categories, results):
    lca = bc.LCA({products[0]: 1}, categories[0])
    lca.lci()
    lca.lcia()
    
    method_matrices = [lca.characterization_matrix.copy()]
    
    for other_method in categories[1:]:
        # Only build each characterization matrix once instead of once per product
        lca.switch_method(other_method)
        method_matrices.append(lca.characterization_matrix.copy())
    
    for i, product in enumerate(products):
        lca.lci({product.id: 1})
        for j, characterization_matrix in enumerate(method_matrices):
            results[i, j] = (characterization_matrix * lca.inventory).sum()
    
    return results

In [ ]:
from time import time

start = time()
results = get_lcia_scores(products_in_order, categories_in_order, results)
print(time() - start)

## Exercise

Compare how long it would take to calculate LCA results for 3 products and 3 impact categories if you had to create a new LCA object each time.

In [ ]:
from scipy import stats

def create_correlation_matrix(scores_array):
    num_methods = scores_array.shape[1]
    correlations = np.zeros((num_methods, num_methods))

    for row in range(num_methods):
        for col in range(num_methods):
            if col <= row:
                continue                               # Only need to compute correlation once
            dataset_1 = scores_array[:, row]
            dataset_2 = scores_array[:, col]
            mask = (dataset_1 != 0) * (dataset_2 != 0) # Ignore activities that have zero score
            corr = stats.kendalltau( # Get tau value, drop p-statistic
                dataset_1[mask], 
                dataset_2[mask]
            )[0]
            if np.isnan(corr):
                correlations[row, col] = 0
            else:
                correlations[row, col] = corr

    correlations = correlations + correlations.T       # Make sorting easier by adding filling in lower left triangle
    return correlations

In [ ]:
correlation_matrix = create_correlation_matrix(results)

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

fig = plt.gcf()
fig.set_size_inches(12, 12)

masked_correlation = np.ma.array(correlation_matrix, mask=correlation_matrix == 0).T
plt.pcolor(masked_correlation, cmap=plt.cm.cubehelix_r)
plt.colorbar()
plt.ylim(None, correlation_matrix.shape[1])
plt.xlim(None, correlation_matrix.shape[0])
plt.xticks(np.arange(0.5, 10), [obj[1] for obj in categories_in_order])
plt.yticks(np.arange(0.5, 10), [obj[1] for obj in categories_in_order])
plt.tight_layout()

In [ ]:
for category in categories_in_order:
    print(category[1], bd.methods[category]['description'])